In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import sys

sys.path.append('..')

from benchmark.himmelblau import Himmelblau
from benchmark.holder import Holder
from benchmark.rosenbrock import Rosenbrock
from benchmark.michalewicz import Michalewicz

from optims.SBS import SBS

In [2]:
def create_bounds(min, max, dim):
    bounds = [(min, max) for _ in range(dim)]
    return np.array(bounds)

In [3]:
fs = {
  "Himmelblau": [Himmelblau(), create_bounds(-4, 4, 2)],
  "Holder Table": [Holder(), create_bounds(-10, 10, 2)],
  "Michalewicz": [Michalewicz(), create_bounds(0, np.pi, 2)],
  "Rosenbrock": [Rosenbrock(), create_bounds(-3, 3, 2)],
}

evals = np.linspace(300, 1_000_000, 10, dtype=int)

def exp(sigma):
  sbs_results = {}
  sbs_noisy_results = {}

  for f_name in tqdm(fs.keys()):
    h = fs[f_name][0]
    bounds = fs[f_name][1]

    noisy_h = lambda x: h(x) + np.random.normal(0, sigma)

    sbs_results_tmp = []
    sbs_noisy_results_tmp = []
    for num_evals in evals:
      svgd_iter = 300
      n_particles = max(num_evals // (300 * 3), 1)
      sbs = SBS(
        bounds,
        n_particles=n_particles,
        k_iter=[10_000],
        svgd_iter=svgd_iter,
        sigma=1 / n_particles**2,
        lr=0.2
      )
      opt = sbs.optimize(h)[0][1]
      sbs_results_tmp.append(np.abs(h.min - opt))

      noisy_opt = sbs.optimize(noisy_h)[0][1]
      sbs_noisy_results_tmp.append(np.abs(h.min - noisy_opt))
      
    sbs_results[f_name] = sbs_results_tmp
    sbs_noisy_results[f_name] = sbs_noisy_results_tmp

  return sbs_results, sbs_noisy_results

In [4]:
sigma = 0.1

sbs_results, sbs_noisy_results = exp(sigma)

  0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
for f_name in fs.keys():
  plt.plot(evals, sbs_results[f_name], label="vanilla")
  plt.plot(evals, sbs_noisy_results[f_name], label="noisy")
  plt.xlabel("Number of evaluations")
  plt.ylabel("log distance to minimum")
  plt.yscale("log")
  plt.title(f"Robustness of SBS on noisy {f_name} $\sigma = 0.1$")
  plt.legend()
  plt.savefig(f"fig/noisy/noisy_01_sbs_{f_name}.png", bbox_inches="tight")
  plt.clf()

<Figure size 640x480 with 0 Axes>

In [6]:
sigma = 0.01

sbs_results, sbs_noisy_results = exp(sigma)

  0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
for f_name in fs.keys():
  plt.plot(evals, sbs_results[f_name], label="vanilla")
  plt.plot(evals, sbs_noisy_results[f_name], label="noisy")
  plt.xlabel("Number of evaluations")
  plt.ylabel("log distance to minimum")
  plt.yscale("log")
  plt.title(f"Robustness of SBS on noisy {f_name} $\sigma = 0.01$")
  plt.legend()
  plt.savefig(f"fig/noisy/noisy_001_sbs_{f_name}.png", bbox_inches="tight")
  plt.clf()

<Figure size 640x480 with 0 Axes>